In [ ]:
library(tidyverse)
library(tidymodels)
library(data.table)
library(zoo)
library(weathermetrics)
library(lubridate)

In [5]:
x2017 <- fread('~/hakmar/2017.csv',dec = '.')
x2018 <- fread('~/hakmar/2018.csv',dec = '.')
x2019 <- fread('~/hakmar/2019.csv',dec = '.')
x2020 <- fread('~/hakmar/2020.csv',dec = '.')
x2021 <- fread('~/hakmar/2021.csv',dec = '.')
colname <- fread('~//hakmar_fulldata.csv',nrows = 1)
data <- list(x2017,x2018,x2019,x2020,x2021) %>% rbindlist
colnames(data) <- names(colname)[2:11]
ozel_gun_anlam <- fread('~/hakmar/Ozel_gunler.csv')
stok <- fread('~//hakmar/Stoklar.csv') %>% mutate(stok_kodu=V1,urun=paste(V17,V2)) %>% select(stok_kodu,urun)

In [6]:
stok  %>% mutate(kategori= apply(stok %>% select(urun),1,kategorize)) -> kategorized_urun

In [48]:
hava <- fread('~/hakmar/temp.csv')
yagmur1 <- fread('~/hakmar/yagmur.csv')
yagmur2 <- fread('~/hakmar/yagmur2.csv')
colnames(yagmur2) <- c('tarih','.xxxx','.xxx','.xx','.x','yagmur')

yagmur1 <- yagmur1 %>%
rename(tarih=date,yagmur=rain_drizzle) %>%
select(tarih,yagmur)

yagmur2 <- yagmur2 %>%
select(tarih,yagmur) %>%
mutate(tarih=as.Date(tarih,'%m/%d/%Y')) %>% 
mutate(yagmur=gsub('[^0-9.]','',yagmur)) %>% 
mutate(yagmur=as.numeric(yagmur)) %>% 
mutate(yagmur=as.integer(ifelse(yagmur>0,1,0)))%>%
mutate(yagmur=na.approx(yagmur)) %>% 
mutate(yagmur=as.integer(ifelse(yagmur>0,1,0)))%>% 
rbind(data.table(tarih=as.Date('2017-01-01'),yagmur=0)) -> yagmur2

max(yagmur1$tarih) + 1 -> max_date
yagmur2 <- yagmur2 %>% filter(tarih>=max_date)

rbind(yagmur1 %>% mutate(tarih=as.Date(tarih)),yagmur2) %>% 
rbind(data.table(tarih=as.Date('2017-01-01'),yagmur=0)) -> yagmur

hava %>%
select(DATE,TAVG) %>%
rename(tarih=DATE,ort_sicaklik=TAVG) %>%
mutate(tarih=as.Date(tarih),ort_sicaklik=fahrenheit.to.celsius(ort_sicaklik)) -> hava

In [14]:
data %>%
mutate(tarih=ymd_hms(tarih)) %>%
left_join(kategorized_urun,by='stok_kodu') %>%
mutate(tur=ifelse(grepl('ACIK',urun),'Acik','Paket')) %>% 
select(-urun) %>% 
mutate(arife=ifelse(ozel_gun==2 | ozel_gun==1,1,0),yilbasi=ifelse(ozel_gun==6,1,0),asure=ifelse(ozel_gun==4,1,0)) %>% 
select(-c(stok_kodu,ozel_gun)) %>% 
group_by(magaza_no,tarih,kategori,tur) %>%
summarise(top_insert=sum(insert),
          top_teshir=sum(teshir),
          arife=ifelse(sum(arife)>0,1,0),
          asure=ifelse(sum(asure)>0,1,0),
          yilbasi=ifelse(sum(yilbasi>0),1,0),
          toplam_satis_miktari=sum(satis_miktari),
          toplam_satis_tutari=sum(satis_tutari)) %>% 
ungroup %>% 
mutate(hafta_gun=wday(tarih),hafta_yil=isoweek(tarih),ay=month(tarih),yil=year(tarih)) %>% 
mutate(trend=ifelse(hafta_yil>40 & hafta_yil<=52,hafta_yil-39,0)) %>%
left_join(yagmur,by='tarih') %>% 
left_join(hava,by='tarih') -> data_

data_ %>%
select(tarih,asure) %>%
filter(asure==1) %>%
select(tarih) %>%
unique %>% 
as.data.table %>%  
slice(rep(1:n(),30)) %>% 
group_by(tarih) %>%
mutate(sira=1:n()) %>% 
ungroup %>% 
mutate(sira=sira-1) %>% 
mutate(tarih=as.Date(tarih)+sira) %>% 
group_by(tarih) %>% 
mutate(asure_trend=-sira-1) %>% 
select(-sira) %>% 
ungroup -> asure_trend_dt 

data_ <- data_ %>% left_join(asure_trend_dt,by='tarih') %>% mutate(asure_trend=ifelse(is.na(asure_trend),0,asure_trend))

`summarise()` has grouped output by 'magaza_no', 'tarih', 'kategori'. You can override using the `.groups` argument.



In [70]:
yagmur %>% 
mutate(yil=year(tarih),hafta_yil=week(tarih)) %>% 
group_by(yil,hafta_yil) %>% 
summarise(haftalik_toplam_yagmur=sum(yagmur)) -> yagmur_aggr

`summarise()` has grouped output by 'yil'. You can override using the `.groups` argument.



In [25]:
data_ %>% fwrite('~/hakmar/hakmar_aggregate.txt')

In [68]:
data_ %>%
group_by(yil,hafta_yil,magaza_no,kategori,tur) %>% 
summarise(top_insert=sum(top_insert),
          top_satis_miktar=sum(toplam_satis_miktari),
          asure=sum(asure),yilbasi=sum(yilbasi),
          arife=sum(arife)) %>% 
ungroup %>% 
mutate(trend_=ifelse(hafta_yil>=40 & hafta_yil<=52,hafta_yil-39,0)) -> data_final

data_final %>% fwrite('~/Desktop/hakmar_tertemiz.txt')

`summarise()` has grouped output by 'yil', 'hafta_yil', 'magaza_no', 'kategori'. You can override using the `.groups` argument.



In [74]:
data_final %>% left_join(yagmur_aggr,by=c('yil','hafta_yil')) %>% na.omit%>% fwrite('~/Desktop/hakmar_tertemiz.txt')

In [75]:
data_final %>% left_join(yagmur_aggr,by=c('yil','hafta_yil')) %>% na.omit -> data_final

In [110]:
data_final %>% mutate(magaza_no=as.factor(magaza_no),
                               asure=as.factor(asure),
                               yilbasi=as.factor(yilbasi),
                               arife=as.factor(arife)) %>% 
filter(tur=='Acik') -> data_final

In [113]:
data_final %>% select(-tur) -> data_final

In [180]:
hakmar_recipe <- recipe(top_satis_miktar ~ .,data = data_final) %>% 
step_corr(all_numeric(),-all_outcomes(), threshold = 0.1) %>% 
step_dummy(all_nominal()) %>% 
prep(data_final) %>% 
bake(data_final)


hakmar_recipe_train <- hakmar_recipe %>% filter(yil==2017 | yil==2018) %>% filter
hakmar_recipe_test <- hakmar_recipe %>% filter(yil==2019)

model <- linear_reg() %>% 
set_engine('lm') %>% 
set_mode('regression') %>% 
fit(top_satis_miktar ~ . -yil,data = hakmar_recipe_train)

In [192]:
model %>% 
predict(hakmar_recipe_test) %>% 
mutate(.pred=ifelse(.pred<0,0,.pred)) %>% 
bind_cols(hakmar_recipe_test %>% select(top_satis_miktar,yil,hafta_yil)) %>% 
group_by(yil,hafta_yil) %>% 
summarise(gercek=sum(top_satis_miktar),
          tahmin=sum(.pred)) %>% 
ungroup %>% 
rmse(gercek,tahmin)



Warning message in predict.lm(object = object$fit, newdata = new_data, type = "response"):
“prediction from a rank-deficient fit may be misleading”


.pred,top_satis_miktar,yil,hafta_yil
<dbl>,<dbl>,<dbl>,<dbl>
69.0694990,25.668,2019,1
0.9995923,3.926,2019,1
317.7276292,357.758,2019,1
27.8436318,18.991,2019,1
115.0029894,62.594,2019,1
333.8463862,367.676,2019,1
0.0000000,2.898,2019,1
318.4297051,275.031,2019,1
25.6644072,4.052,2019,1


In [196]:
model <- lm(Sepal.Length~.,iris)
summary(model)

library(broom)

tidy(model)


Call:
lm(formula = Sepal.Length ~ ., data = iris)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.79424 -0.21874  0.00899  0.20255  0.73103 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)        2.17127    0.27979   7.760 1.43e-12 ***
Sepal.Width        0.49589    0.08607   5.761 4.87e-08 ***
Petal.Length       0.82924    0.06853  12.101  < 2e-16 ***
Petal.Width       -0.31516    0.15120  -2.084  0.03889 *  
Speciesversicolor -0.72356    0.24017  -3.013  0.00306 ** 
Speciesvirginica  -1.02350    0.33373  -3.067  0.00258 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3068 on 144 degrees of freedom
Multiple R-squared:  0.8673,	Adjusted R-squared:  0.8627 
F-statistic: 188.3 on 5 and 144 DF,  p-value: < 2.2e-16


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),2.1712663,0.27979415,7.760227,1.429502e-12
Sepal.Width,0.4958889,0.08606992,5.761466,4.867516e-08
Petal.Length,0.8292439,0.06852765,12.100867,1.073592e-23
Petal.Width,-0.3151552,0.15119575,-2.084418,3.888826e-02
Speciesversicolor,-0.7235620,0.24016894,-3.012721,3.059634e-03
Speciesvirginica,-1.0234978,0.33372630,-3.066878,2.584344e-03


In [197]:
install.packages('stargazer')

Installing package into ‘/home/samet/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)



In [ ]:
stargazer(mylist$fits, coef =  mylist$coefs,
              se = mylist$ses,
              type = m_type, title = title_str, 
              out = paste0("~/projects/outputs",fname),  single.row = T ,...)

In [200]:
library(stargazer)


Please cite as: 


 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 




In [211]:
set.seed(12345)
n = 100
x1 = rnorm(n)
x2 = rnorm(n)+0.1
y = x1 + rnorm(n)

df <- data.frame(x1, x2, y)
x_str <- c("x1", "x1+x2")

regf_lm <- function(df,y_var, x_str ) {
  frmla <- formula(paste0(y_var," ~ ", x_str ))
  fit <- lm(frmla, data = df ) 
  return(list(fit,select(tidy(fit),std.error))) #fit
}

exm_model <- regf_lm(iris,'Sepal.Width','Sepal.Length')

stargazer(exm_model[[1]], coef =  exm_model[[2]], title = 'x_model', 
              out ='abc',  single.row = T)



% Table created by stargazer v.5.2.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Pzt, Nis 26, 2021 - 09:05:13
\begin{table}[!htbp] \centering 
  \caption{x_model} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{1}{c}{\textit{Dependent variable:}} \\ 
\cline{2-2} 
\\[-1.8ex] & Sepal.Width \\ 
\hline \\[-1.8ex] 
 Sepal.Length & 0.043 (0.043) \\ 
  Constant & 0.254 (0.254) \\ 
 \hline \\[-1.8ex] 
Observations & 150 \\ 
R$^{2}$ & 0.014 \\ 
Adjusted R$^{2}$ & 0.007 \\ 
Residual Std. Error & 0.434 (df = 148) \\ 
F Statistic & 2.074 (df = 1; 148) \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{1}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 


In [214]:
structure(c(0.122500582083785, 0.29427544459643, 0.223506623031398, 
0.305543938017955, 0.3555346545493, 0.13276591893586, 0.29851456949357, 
0.226163385898154, 0.305334527655609, 0.349099920643732, 0.142765775203754, 
0.302517405153014, 0.228743825400136, 0.305071530853091, 0.343012833399477, 
0.152500150887466, 0.30628395157476, 0.231247941537343, 0.304754947610403, 
0.337273392816537, 0.161969045986996, 0.309814208758809, 0.233675734309774, 
0.304384777927542, 0.331881598894911, 0.171172460502345, 0.313108193745883, 
0.236027203717432, 0.30396104662241, 0.326837451634599, 0.180110394433513, 
0.316165957658146, 0.238302349760315, 0.303483828148701, 0.322140951035602, 
0.188782847780499, 0.318987602739927, 0.240501172438425, 0.302953256861566, 
0.317791977513919, 0.197189820543304, 0.321573299398441, 0.242623671751759, 
0.302369537283266, 0.313790172317547, 0.205331312721925, 0.323923303244507, 
0.244669847700318, 0.301732954368829, 0.310134817942479, 0.213207324316361, 
0.326037972133275, 0.246639700284097, 0.301043883771709, 0.306824677535159, 
0.220817855326607, 0.327917783204944, 0.248533229503092, 0.300302802109442, 
0.30385783429494, 0.228162905752658, 0.329563349925485, 0.250350435357295, 
0.299510128025495, 0.301231530876531, 0.235242475594506, 0.330975439127363, 
0.252091317846694, 0.298666063251117, 0.298942008792457, 0.242056564852138, 
0.332154988050262, 0.253755876971275, 0.297770433667191, 0.296984347815508, 
0.248605173525543, 0.3331031213818, 0.255344077035364, 0.296822530366082, 
0.295352305381194, 0.254888301614705, 0.333821168298262, 0.25685581095197, 
0.295820950713493, 0.294038155990196, 0.260905949119606, 0.334310679505311, 
0.258290864547128, 0.294763718181327, 0.29303253061082, 0.266658116040223, 
0.334573444278715, 0.259648880864242, 0.293648402180558, 0.292324256081446, 
0.272144802376533, 0.334611507505068, 0.260929324468427, 0.292472237894095, 
0.29190019451298, 0.277366008128505, 0.334427186722508, 0.262131445750849, 
0.291232246109652, 0.29174545781661, 0.282321733296109, 0.334023089161438, 
0.263254245233068, 0.289925353052612, 0.291843622231568, 0.287011977879309, 
0.33340212878525, 0.264296437871382, 0.288548510218893, 0.29217694285288, 
0.291436741878066, 0.332567543331044, 0.265256417361163, 0.28709881420782, 
0.292726568159131, 0.295596025292336, 0.331522911350348, 0.266132220441202, 
0.285573626554987, 0.293472754540217, 0.299489828122074, 0.330272169249841, 
0.266921653895772, 0.283970693565133, 0.294395080825109, 0.30311815036723, 
0.328819628332072, 0.267622421556679, 0.282288266145008, 0.295472662809605, 
0.306480992027753, 0.32716999183618, 0.268232251305332, 0.280525371105845, 
0.296684367784092, 0.309578353103588, 0.325328371978616, 0.268749022074797, 
0.278682082465839, 0.298009029061305, 0.31241023359468, 0.323300306993861, 
0.269170890851861, 0.276759792752618, 0.299425660504082, 0.314976633500972, 
0.321091778175144, 0.269496360401606, 0.27476145142608, 0.300913671053126, 
0.317277552822408, 0.318709226915169, 0.269724346991979, 0.272691803301225, 
0.30245307925476, 0.319312991558926, 0.316159571746827, 0.269854177560595, 
0.270557267102268, 0.304024727788694, 0.321082949710463, 0.313450225383922, 
0.269885586881538, 0.268365814016749, 0.305610497995776, 0.32258742727695, 
0.31058911176189, 0.26981871473216, 0.266126846249641, 0.307193524405759, 
0.323826424258312, 0.307584683078521, 0.269654103059881, 0.263851075577464, 
0.308758409265061, 0.324799940654468, 0.304445936834679, 0.269392693148992, 
0.261550401902396, 0.310291437064519, 0.325507976465329, 0.301182432875025, 
0.269035822787454, 0.259237720941508, 0.311780408602389, 0.325950687955043, 
0.297804310428734, 0.268585223433699, 0.256926731916016, 0.313214475047339, 
0.326128543916233, 0.294322305150219, 0.268043017383431, 0.254631745240517, 
0.31458397200144, 0.326042481934244, 0.290747766159853, 0.267411784894658, 
0.252367490212232, 0.315880326567261, 0.325694064651384, 0.287092673084685, 
0.266694631312714, 0.25014892270025, 0.317095964414956, 0.325085636031168, 
0.283369653099159, 0.26589517822546, 0.247991032834772, 0.318224216849358, 
0.324220477622561, 0.279591997965842, 0.265017554618472, 0.245908652696348, 
0.319259227877065, 0.323102964824218, 0.275773681076135, 0.264066388030237, 
0.243916264005123, 0.320195861273532, 0.321738723148725, 0.271929374491003, 
0.263046795707343, 0.242027805810076, 0.321029607650156, 0.320134784486837, 
0.268074465981687, 0.261964375759676, 0.240256482178265, 0.321756491521366, 
0.318299743371724, 0.264225076070435, 0.260825198315607, 0.238614569884064, 
0.322372962063665, 0.316243679297106, 0.260398075071212, 0.259635796677185, 
0.237113226098407, 0.322875783874669, 0.313978079035401, 0.256611100130432, 
0.258403158475333, 0.235762296078024, 0.323261927732155, 0.311515758955865, 
0.252882572267672, 0.257134716825033, 0.234570241267049, 0.3235284613531, 
0.308870787342738, 0.249231713416398, 0.255838341480526, 0.233544067398609, 
0.323672440152728, 0.306058406713381, 0.245678349309024, 0.254522329990498, 
0.232689252596435, 0.323690845289486, 0.303094956136427, 0.242242712361977, 
0.253195398853274, 0.232009675476455, 0.323580521710023, 0.299997793549915, 
0.238945244560758, 0.251866674672007, 0.231507543248398, 0.323338116194172, 
0.29678521807944, 0.23580651752849, 0.250545685309873, 0.231183394176578, 
0.322960282841732, 0.293476392356293, 0.232847152594459, 0.24924235104526, 
0.231036100040681, 0.322443888559247, 0.290091264835605, 0.230087740862665, 
0.247966975726959, 0.231062868596549, 0.321786334994613, 0.286650492114492, 
0.227548763280363, 0.246730237929358, 0.231259246036969, 0.320985880471676, 
0.283175361250197, 0.225250510706613, 0.245543182107633, 0.231619119452455, 
0.320041961924843, 0.279687712078233, 0.223213003980825, 0.244417209752942, 
0.232134945254056, 0.318955516833674, 0.276209859530533, 0.221455913991301, 
0.243364070547614, 0.232797977596153, 0.317729305157494, 0.272764515953584, 
0.219998481743785, 0.242395853520345, 0.233598666990459, 0.316368231269988, 
0.269374713426581, 0.218859438430006, 0.241524978201387, 0.234527058920018, 
0.31487966589381, 0.266063726079564, 0.218056925496229, 0.240764185777739, 
0.235573192453196, 0.313273768035183, 0.262854992411564, 0.217608414711796, 
0.240126530248332, 0.236727498857683, 0.311563535534158, 0.259772037608749, 
0.217530628237682, 0.239625369579227, 0.237981200214491, 0.309764855614871, 
0.256838395862566, 0.217839458695038, 0.239274356858795, 0.239326708031943, 
0.307896555435795, 0.254077532687885, 0.21854988923374, 0.23908743145291, 
0.240758021859681, 0.305980452640002, 0.251512767241148, 0.219675913600942, 
0.239078810160136, 0.242271127902658, 0.304041405905414, 0.249167194638505, 
0.221230456209624, 0.239262978366919, 0.243864397635134, 0.302107009527295, 
0.247063608273964, 0.223225292207137, 0.239654681202772, 0.245538986414679, 
0.300207288000738, 0.24522442213753, 0.225670967543757, 0.240268914695466, 
0.247298640640783, 0.298374390603155, 0.243671593133347, 0.22857671904123, 
0.241120916926218, 0.249149504913477, 0.296642285976767, 0.242426543397849, 
0.231950394461322, 0.242226159184876, 0.251099929191946, 0.295046456711089, 
0.241510082617894, 0.235798399150704, 0.24360035674759, 0.253160275953148, 
0.293623651402667, 0.24094233034892, 0.240125642685834, 0.245259479654476, 
0.255342727350427, 0.292411636714809, 0.240742638333078, 0.244935485517846, 
0.24721961188219, 0.257661042038091, 0.291448949437321, 0.240929512817386, 
0.250229685617427, 0.24949681051651, 0.260130311995991, 0.290774648546243, 
0.24152053687187, 0.256008345119707, 0.252106964924913, 0.262766719354095, 
0.290428067263577, 0.242532292707707, 0.262269856969141, 0.255065597725271, 
0.265587293217067, 0.290448642406514, 0.243980283995372, 0.269010851564392, 
0.258387665754552, 0.268609666488846, 0.290875743736649, 0.245878858182779, 
0.276226143403217, 0.262087361037516, 0.271851852897748, 0.291748449056557, 
0.248241128813427, 0.28390867772735, 0.266177911755412, 0.275332024021565, 
0.293105319306367, 0.251078897844541, 0.292049477167391, 0.270671383214678, 
0.279068286312674, 0.294984173660331, 0.254402577965214, 0.300637588387686, 
0.275578478815637, 0.283078504775003, 0.2974218646234, 0.258221114914552, 
0.309660028731211, 0.280908341021193, 0.287380126641007, 0.300454053127795, 
0.262541909799814, 0.31910173286446, 0.286668504239361, 0.291989965731501, 
0.30411446267131, 0.267370746373676, 0.328945499422319, 0.292864847705798, 
0.296923986815492, 0.30843413345561, 0.272711718311492, 0.339171937652958, 
0.299501548366326, 0.302197089970019, 0.313440676524529, 0.278567156488555, 
0.349759414062702, 0.306581033759465, 0.307822899506313, 0.319157741580355, 
0.284937556257366, 0.360683999060919, 0.314103934898963, 0.313813552895955, 
0.325604484800096, 0.291821504724888, 0.371919413604895, 0.322069039156317, 
0.320179584601277, 0.332795036651765, 0.299215608029818, 0.383436975844722, 
0.330473243143307, 0.326929809905745, 0.340737969710648, 0.307114418619842, 
0.39520554776817, 0.339311505594524, 0.334071208744359, 0.349435766475587, 
0.315510362528903, 0.407191760639708, 0.348576800249894, 0.341608809534039, 
0.358884287185246, 0.324393666654463, 0.419360240439517, 0.358260068737214, 
0.349545573004019, 0.369072237634391, 0.333752286034761, 0.431673833302506, 
0.368349947236391, 0.357882146871665, 0.379981188814486, 0.343571831126079, 
0.444093830957324, 0.378832493143691, 0.366616620518293, 0.391585596554292, 
0.353835495080004, 0.456580196165372, 0.389690911735983, 0.375744279664984, 
0.403852821160465, 0.364523981020685, 0.469091788159823, 0.400905282834985, 
0.385257361048402, 0.41674314705816, 0.375615429322105, 0.481586588084624, 
0.412452287471507, 0.395144807096607, 0.430209802431625, 0.387085344885332, 
0.494022144235502, 0.424304934549701, 0.405392020604876, 0.444199622459691, 
0.398906524415788, 0.506356017300949, 0.436432287511301, 0.415980619411518, 
0.458653712551252, 0.411048983700505, 0.518546225603223, 0.448799190999874, 
0.426888191073691, 0.473508111580747, 0.423479884885389, 0.530551690339338, 
0.461365997525064, 0.438088287196747, 0.488694455123648, 0.436163463752476, 
0.542332680822063, 0.474088294126842, 0.449550417763573, 0.50414063869194, 
0.44906095699719, 0.553851259720906, 0.486917396364983, 0.461240409679166, 
0.519771824184084, 0.462130985504474, 0.565071728303117, 0.499800842308543, 
0.473120765315209, 0.535511446334971, 0.475329937624914, 0.575961071674682, 
0.512682886525335, 0.485151021054642, 0.551282219165886, 0.488612352450864, 
0.58648940402131, 0.525505553112053, 0.497288211840346, 0.567007142434468, 
0.501931452135785, 0.596630413849435, 0.538209688724392, 0.50948733572382, 
0.582610508084661, 0.515239674213582, 0.60636139003791, 0.550736300836054, 
0.521701869524086, 0.598018906696684, 0.528489203917942, 0.615663247889713, 
0.563027895997754, 0.533884284486589, 0.613162233936982, 0.541632506501669, 
0.624520555183645, 0.575029818096232, 0.545986561942096, 0.62797469700819, 
0.554622859556019, 0.632921363306624, 0.586691586613256, 0.557960708965598, 
0.64239582109909, 0.567414885330036, 0.640857038385973, 0.597968234884632, 
0.569759274035213, 0.656371455834571, 0.579965083049889, 0.648322092421715, 
0.608821037724088, 0.581335862691087, 0.669853628939208, 0.592232037466805, 
0.655314014418856, 0.619218239047168, 0.592645653194291, 0.682800399900844, 
0.604176627405, 0.66183310151968, 0.629135779495122, 0.603645912185725, 
0.695175713634172, 0.615762234309616, 0.667882290136042, 0.638556885172412, 
0.614296510345013, 0.706949254144314, 0.626955006714991, 0.673466987081649, 
0.647471656384222, 0.624560438049403, 0.718096298190403, 0.637724124712923, 
0.678594900704359, 0.655876656373961, 0.634403935891786, 0.728597568949157, 
0.648042064420941, 0.68327587201847, 0.663774500060775, 0.643796625198717, 
0.738439089678461, 0.657884862450568, 0.687521705837011, 0.671173442777049, 
0.652711638548437, 0.747612037380951, 0.667232380375592, 0.691346001904038, 
0.678086779639134, 0.661125750288897, 0.756112596467591, 0.676068569200333, 
0.69476398602693, 0.684532244918065, 0.669019507055774, 0.763941812421256, 
0.684381733827911, 0.697792341208681, 0.690531411410286, 0.676377358290497, 
0.771105211180713, 0.692164251452349, 0.700449038780193, 0.696109089808376, 
0.683187786758266, 0.777612418524603, 0.699412289950674, 0.702753169532575, 
0.701292728071771, 0.689443439066075, 0.783476779455426, 0.706125526275022, 
0.704724774849438, 0.706111810797492, 0.695141256180736, 0.78871497758353, 
0.712306864844739, 0.706384677839182, 0.710597258590863, 0.700282603946898, 
0.793346654511093, 0.717962155938483, 0.707754314467297, 0.714780827436241, 
0.704873109639486, 0.797394029216109, 0.723099914086325, 0.708855564688656, 
0.718694508067734, 0.708922498516128, 0.800881517436373, 0.727731036461855, 
0.709710583579807, 0.722369925339919, 0.71244443036959, 0.80383535105347, 
0.731868521274286, 0.710341678281062, 0.725837737598569, 0.715456336080209, 
0.806283197476759, 0.735527186160559, 0.710771184938589, 0.729127036051374, 
0.717979254168325, 0.808253779027357, 0.738723386577443, 0.711021345646504, 
0.732264744138663, 0.720037399124374, 0.80977649232213, 0.741474877412173, 
0.711114185388963, 0.735275016904129, 0.721657729738968, 0.810881027657683, 
0.743800674593081, 0.711071388982251, 0.738178640365557, 0.72286951743299, 
0.811596988394345, 0.745720916700233, 0.710914178016879, 0.740992430885546, 
0.723703914587683, 0.811953510340171, 0.747256726576058, 0.710663187799672, 
0.74372863454224, 0.72419352287474, 0.811978881134926, 0.748430072935988, 
0.710338344295869, 0.746395735886435, 0.724371961586399, 0.81170053093731, 
0.749263631979087, 0.709958741071214, 0.748999266698697, 0.724273435965527, 
0.811145023112168, 0.749780648998687, 0.709542516234057, 0.751542614746468, 
0.723932305535715, 0.810338044917707, 0.750004799993021, 0.709106813027563, 
0.754027337698552, 0.723382858017655, 0.809304398192715, 0.749960053275859, 
0.708667740421929, 0.756453477039594, 0.722659083245517, 0.808067990043771, 
0.749670531087139, 0.708240333706598, 0.75881982181071, 0.721794447083327, 
0.80665182353246, 0.749160371203606, 0.707838515082475, 0.761124172350113, 
0.720821665341347, 0.805077988362584, 0.748453588549437, 0.707475054254144, 
0.763363604033752, 0.719772477692451, 0.803367651567383, 0.74757393680688, 
0.707161529022082, 0.765534731015939, 0.718677421588503, 0.80154104819675, 
0.74654477002688, 0.706908285874879, 0.767633969969982, 0.717565606176741, 
0.79961747200445, 0.745388904239713, 0.70672440058145, 0.769657803828819, 
0.716464486216157, 0.797615232745181, 0.744128479065612, 0.706617638783253, 
0.771603045525647, 0.715399635993871, 0.795551613471647, 0.742784819325399, 
0.706594416586505, 0.77346710173455, 0.714394523241515, 0.793442827831626, 
0.741378296651114, 0.706659761154392, 0.775247852389183, 0.713470144896467, 
0.791303977365038, 0.739928191096637, 0.706817271299289, 0.776943530201444, 
0.712644662863096, 0.789149008801012, 0.73845255274832, 0.707069078074972, 
0.778552600180157, 0.711933039773997, 0.786990758183217, 0.736968063335613, 
0.707415805368835, 0.780073639149748, 0.711346674751237, 0.784840994995184, 
0.735489897841694, 0.707856530494112, 0.781505215268923, 0.710893039167593, 
0.782710466285638, 0.734031586114093, 0.708388744782098, 0.782845767549351, 
0.710575988938223, 0.78060894079382, 0.732605075889282, 0.709008314174368, 
0.784093485374338, 0.710396076812338, 0.778545253074824, 0.731220795817246, 
0.709709439814997, 0.785246188017508, 0.710350864664868, 0.776527347624918, 
0.729887718486064, 0.710484618642787, 0.786301204161482, 0.71043523578814, 
0.774562323006881, 0.728613423446487, 0.711324603983489, 0.787255251416557, 
0.710641707183546, 0.772656475975333, 0.727404160236508, 0.712218555103088, 
0.78810431583938, 0.710960819895194, 0.770815190475517, 0.726264911405942, 
0.7131541867611, 0.788843531451631, 0.71138152934357, 0.769042826642096, 
0.725199455540997, 0.714117918763862, 0.789467409777887, 0.71189159565919, 
0.767342609797934, 0.724210430288856, 0.715095025517813, 0.789970069383483, 
0.712477974016255, 0.765716519452897, 0.723299395382244, 0.716069785582798, 
0.790345465412374, 0.71312720496631, 0.764165178302632, 0.722466895664008, 
0.717025631225352, 0.790587619124998, 0.713825630621193, 0.762687741227368, 
0.721712524111689, 0.717945297972, 0.790690847436135, 0.714559610835998, 
0.761281784290697, 0.721034984862097, 0.718811310593743, 0.790649878840733, 
0.715315739392031, 0.759943193738371, 0.720432156235881, 0.71960646909055, 
0.790459969339718, 0.716081060179766, 0.758666054997089, 0.719901153762111, 
0.720314334675844, 0.790117018365815, 0.716843283381805, 0.757442541673293, 
0.719438393202841, 0.720919373340759, 0.789617684709366, 0.717591001655842, 
0.756263111717257, 0.719039653577692, 0.721407099418396, 0.788959502444147, 
0.71831390631762, 0.755116703587182, 0.718700140188424, 0.721764219148072, 
0.788140996853189, 0.719002843823386, 0.753990932413287, 0.718414547643507, 
0.721978774239583, 0.787161800354588, 0.719649872252329, 0.752872286161905, 
0.718177122882703, 0.722040285437457, 0.786022768427323, 0.720248317789026, 
0.751746321799579, 0.717981728201636, 0.721939896085211, 0.784726326547061, 
0.720792935367653, 0.750598088433132, 0.717821912555189, 0.721670515689609, 
0.783276817121947, 0.721280069316188, 0.749412550449746, 0.717690982860895, 
0.72122696348492, 0.781680233856119, 0.721707708350468, 0.748174765453578, 
0.71758207530233, 0.72060611199717, 0.779943956113207, 0.722075540568228, 
0.74687006220239, 0.717488226632511, 0.719807030608406, 0.778076483279837, 
0.722385008443152, 0.745484218544168, 0.717402445477285, 0.718830817523893, 
0.776087339664835, 0.722639301829306, 0.744003723368089, 0.71731778363873, 
0.717680431739323, 0.773986979398426, 0.722843350965568, 0.74241603855547, 
0.717227407398546, 0.716360351478335, 0.771786602362918, 0.723003480796795, 
0.740709646439553, 0.717124668821455, 0.714876232630052, 0.769497970123383, 
0.723127065295006, 0.738874097265286, 0.717003177058603, 0.713234567186606, 
0.767133221858344, 0.723222181780549, 0.736900056649105, 0.716856869650955, 
0.711442672084187, 0.764704741356475, 0.72329726524328, 0.734779353038717, 
0.716680052279332, 0.709508678044102, 0.762225024013313, 0.723360762663733, 
0.732505025172883, 0.716467438514453, 0.707441518413822, 0.759706543827961, 
0.723420787334297, 0.730071369541197, 0.716214189566969, 0.705250918008038, 
0.757161620399798, 0.723484773180396, 0.727473987843868, 0.715915954037502, 
0.702947381949723, 0.754602285925185, 0.723559129081655, 0.724709834451509, 
0.715568907666681, 0.700542189726354, 0.752040152194173, 0.723649155534245, 
0.721777263864922, 0.715169793085183, 0.698047389246153, 0.749486277587218, 
0.723758961313215, 0.718676078174886, 0.714715959563765, 0.69547553236811, 
0.746951034071887, 0.723891150659456, 0.715407574521941, 0.714205113785683, 
0.692839410432002, 0.744443974199575, 0.724046510466666, 0.711974592556176, 
0.713635070619101, 0.690151789788423, 0.741973698102213, 0.724223697468319, 
0.708381561897018, 0.713003503889507, 0.687425442217445, 0.739547945199311, 
0.724419541802812, 0.704634761535462, 0.712307945736596, 0.684673175347285, 
0.737173685904985, 0.72462935057862, 0.700742579236306, 0.71154578597115, 
0.681907807390831, 0.734857078671981, 0.724847170076432, 0.696715515822678, 
0.710714271431922, 0.679142141882164, 0.732603427035687, 0.725066047951282, 
0.692566189460571, 0.709810505342516, 0.676388942413085, 0.730417136658153, 
0.725278295434689, 0.68830933994337, 0.708831446668265, 0.673660907369642, 
0.728301801609538, 0.725475749536789, 0.683961532664851, 0.707773909473119, 
0.670970644668651, 0.726260290649554, 0.725650035248473, 0.679540862592175, 
0.706634562276514, 0.668330646494225, 0.724294833508911, 0.725792618793141, 
0.675066632665642, 0.70541005745808, 0.665753264034301, 0.722407107170761, 
0.725894860878465, 0.670559032198432, 0.704097160662347, 0.663250682217163, 
0.720598322152139, 0.725948069948145, 0.666038815276351, 0.70269288020344, 
0.660834894447964, 0.718869361333807, 0.725943555433669, 0.661526979157582, 
0.701194596469787, 0.658517677345255, 0.71722091879009, 0.725872681006075, 
0.657044442672429, 0.699600191328815, 0.656310432384337, 0.715653486201941, 
0.725726917827706, 0.652611724623066, 0.697908177531652, 0.654224027540619, 
0.71416733927, 0.725497897803973, 0.648248622183291, 0.696117828117821, 
0.652268638932969, 0.712762524127661, 0.725177466835113, 0.643973889298266, 
0.694229305819949, 0.650453592467073, 0.71143884375413, 0.724757738067951, 
0.639805169201532, 0.692243792468457, 0.648787205478782, 0.710195844387489, 
0.724231145147664, 0.635758926932014, 0.690163618396267, 0.647276628377471, 
0.709032561162971, 0.723590705910877, 0.631850381851025, 0.687992202536576, 
0.64592768628939, 0.707947263751228, 0.722830286078761, 0.62809344015928, 
0.685733992520634, 0.644744720701018, 0.706937201996604, 0.721944862950135, 
0.624500627413897, 0.683394404775524, 0.643730723735348, 0.705998711747714, 
0.720930789094558, 0.621083251549228, 0.680979764621941, 0.642887472428173, 
0.705127320688011, 0.719786056045427, 0.617851565897687, 0.678497246371966, 
0.642215663004369, 0.70431785416636, 0.718510563041713, 0.614814907883497, 
0.675954813426841, 0.641715045154181, 0.703564541027608, 0.7171063857697, 
0.611981837716444, 0.673361158374748, 0.64138455630951, 0.70286111944316, 
0.71557804510472, 0.609360277085623, 0.67072564308858, 0.641222455920195, 
0.702200942741548, 0.713932302025208, 0.60695747626894, 0.668058238823713, 
0.641226459730308, 0.701577085239013, 0.712177952526762, 0.604779981242614, 
0.66536946631578, 0.641393874054435, 0.700982448070075, 0.710325622536197, 
0.602833600790677, 0.662670309668224, 0.641721730053974, 0.700409865018109, 
0.708387562825601, 0.601123373614478, 0.659972130239842, 0.642206918013419, 
0.699852208345925, 0.706377443926391, 0.599653535442188, 0.65728658053234, 
0.642846321616653, 0.699302720450811, 0.704310202432958, 0.598427388074301, 
0.654625518077879, 0.643636952223237, 0.698755345519584, 0.702201887306308, 
0.597447168429141, 0.652000919326621, 0.644575788013852, 0.69820487717238, 
0.700069350352095, 0.59671391758836, 0.649424793534284, 0.645659613135754, 
0.697647106106444, 0.697929936698657, 0.596227349842446, 0.646909096649695, 
0.646884856848223, 0.697078967739922, 0.695801175275049, 0.595985721736223, 
0.64446564520234, 0.648247464825912, 0.696498492437792, 0.693700469289074, 
0.595985839720184, 0.642106030189918, 0.649742770462198, 0.695904755737797, 
0.691644786705316, 0.596223067801817, 0.639841530965896, 0.651365164621128, 
0.69529782857638, 0.689650350723165, 0.596691116120784, 0.63768288951685, 
0.65310776538936, 0.694678727514616, 0.687732330254854, 0.597381829524094, 
0.635640084739809, 0.654962087828103, 0.694049364964146, 0.685904530403488, 
0.598284976141281, 0.633722106719599, 0.656918021910071, 0.693412499413118, 
0.684179438377916, 0.59938838860409, 0.631936979568404, 0.658963810456427, 
0.692771685652114, 0.6825682694076, 0.600678105266165, 0.630291784265325, 
0.661086027073734, 0.692131225000089, 0.681081012657484, 0.602138707472509, 
0.62879268149594, 0.663269554090901, 0.691496115530307, 0.679726477142865, 
0.603753656828942, 0.627444934491862, 0.665497560496134, 0.690872002296272, 
0.67851233764427, 0.605505632471562, 0.626252931870302, 0.667751479873891, 
0.690264975936618, 0.677445180622324, 0.607376659589943, 0.625220210473622, 
0.670010988341832, 0.689681371279992, 0.676530550132623, 0.609348237950327, 
0.624349478208897, 0.672253982487778, 0.689127565949943, 0.675772791530688, 
0.611401470418823, 0.623642491150168, 0.674456557306661, 0.688609778969806, 
0.675174895176909, 0.6135171914846, 0.623099930640708, 0.676592984137485, 
0.688133869367594, 0.674738340141494, 0.615676095783084), .Dim = c(5L, 
250L)) -> a

In [215]:
a

0.1225006,0.1327659,0.1427658,0.1525002,0.1619690,0.1711725,0.1801104,0.1887828,0.1971898,0.2053313,⋯,0.6337221,0.6319370,0.6302918,0.6287927,0.6274449,0.6262529,0.6252202,0.6243495,0.6236425,0.6230999
0.2942754,0.2985146,0.3025174,0.3062840,0.3098142,0.3131082,0.3161660,0.3189876,0.3215733,0.3239233,⋯,0.6569180,0.6589638,0.6610860,0.6632696,0.6654976,0.6677515,0.6700110,0.6722540,0.6744566,0.6765930
0.2235066,0.2261634,0.2287438,0.2312479,0.2336757,0.2360272,0.2383023,0.2405012,0.2426237,0.2446698,⋯,0.6934125,0.6927717,0.6921312,0.6914961,0.6908720,0.6902650,0.6896814,0.6891276,0.6886098,0.6881339
0.3055439,0.3053345,0.3050715,0.3047549,0.3043848,0.3039610,0.3034838,0.3029533,0.3023695,0.3017330,⋯,0.6841794,0.6825683,0.6810810,0.6797265,0.6785123,0.6774452,0.6765306,0.6757728,0.6751749,0.6747383
0.3555347,0.3490999,0.3430128,0.3372734,0.3318816,0.3268375,0.3221410,0.3177920,0.3137902,0.3101348,⋯,0.5993884,0.6006781,0.6021387,0.6037537,0.6055056,0.6073767,0.6093482,0.6114015,0.6135172,0.6156761
